# Aladdin Awesome Session 

Source:

Paulo Cortez, University of Minho, GuimarÃ£es, Portugal, http://www3.dsi.uminho.pt/pcortez


Data Set Information:

This data approach student achievement in secondary education of two Portuguese schools. The data attributes include student grades, demographic, social and school related features) and it was collected by using school reports and questionnaires. Two datasets are provided regarding the performance in two distinct subjects: Mathematics (mat) and Portuguese language (por). In [Cortez and Silva, 2008], the two datasets were modeled under binary/five-level classification and regression tasks. Important note: the target attribute G3 has a strong correlation with attributes G2 and G1. This occurs because G3 is the final year grade (issued at the 3rd period), while G1 and G2 correspond to the 1st and 2nd period grades. It is more difficult to predict G3 without G2 and G1, but such prediction is much more useful (see paper source for more details).


Attribute Information:

# Attributes for both student-mat.csv (Math course) and student-por.csv (Portuguese language course) datasets:
1. school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira)
2. sex - student's sex (binary: 'F' - female or 'M' - male)
3. age - student's age (numeric: from 15 to 22)
4. address - student's home address type (binary: 'U' - urban or 'R' - rural)
5. famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3)
6. Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart)
7. Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
8. Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education)
9. Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
10. Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
11. reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
12. guardian - student's guardian (nominal: 'mother', 'father' or 'other')
13. traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. failures - number of past class failures (numeric: n if 1<=n<3, else 4)
16. schoolsup - extra educational support (binary: yes or no)
17. famsup - family educational support (binary: yes or no)
18. paid - extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities - extra-curricular activities (binary: yes or no)
20. nursery - attended nursery school (binary: yes or no)
21. higher - wants to take higher education (binary: yes or no)
22. internet - Internet access at home (binary: yes or no)
23. romantic - with a romantic relationship (binary: yes or no)
24. famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime - free time after school (numeric: from 1 - very low to 5 - very high)
26. goout - going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health - current health status (numeric: from 1 - very bad to 5 - very good)
30. absences - number of school absences (numeric: from 0 to 93)

# these grades are related with the course subject, Math or Portuguese:
31. G1 - first period grade (numeric: from 0 to 20)
31. G2 - second period grade (numeric: from 0 to 20)
32. G3 - final grade (numeric: from 0 to 20, output target)

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.plotting import plot_decision_regions
from sklearn import tree
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
from pandas.plotting import parallel_coordinates


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv('/kaggle/input/student-grade-prediction/student-mat.csv', engine='python', header=0)
df.head()

## Replace colums with binary data


In [ ]:
sexDummies = pd.get_dummies(df['sex'], prefix = 'sex')
parentDummies = pd.get_dummies(df['Pstatus'], prefix = 'parent')
fJobDummies = pd.get_dummies(df['Fjob'], prefix = 'fjob')
df = pd.concat([df, sexDummies, parentDummies, fJobDummies], axis=1)
df = df.drop(["sex", "Pstatus", 'Fjob'], axis=1)
df.head()


In [ ]:
def getGoodGrade(g3, med):
    if(g3 > med): 
        return 1
    return 0

med = df['G3'].median();
df['grade'] = df.apply(lambda x: getGoodGrade(x['G3'], med), axis=1)
print(med)

In [ ]:
plt.figure(figsize=(5,5))
plt.tight_layout()
seabornInstance.distplot(df['grade'])

In [ ]:
plt.figure(figsize=(5,5))
plt.tight_layout()
seabornInstance.distplot(df['G3'])

In [ ]:
plt.figure(figsize=(5,5))
plt.tight_layout()
seabornInstance.distplot(df['absences'])

In [ ]:


binaryValues = df[["sex_M", "sex_F", "parent_A", "parent_T", "fjob_at_home", "failures", "fjob_at_home" , "fjob_health",  "fjob_other" , "fjob_services" , "fjob_teacher"]]
nominalValues = df[["Fedu", "Medu", "absences"]]
nominal_normValues=(nominalValues-nominalValues.min())/(nominalValues.max()-nominalValues.min())

X = pd.concat([binaryValues, nominal_normValues], axis=1)
y = df['grade']
X.head()

# Linear Regression

In [ ]:
y = df['G3']
X = pd.concat([nominalValues, binaryValues], axis=1);
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
print('(r2) score: %.3f'
      % (r2_score(y_test, y_pred)))
print('score: %.3f'
      % (model.score(X_test, y_test)))

In [ ]:
plt.scatter(X['Fedu'].values, y.values, color='g', s=2)
plt.scatter(X_test['Fedu'].values, y_pred, color='b')
plt.show()

# KNeighbors

In [ ]:
y = df['grade']
X = binaryValues;
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = KNeighborsClassifier(n_neighbors=5) 
model.fit(X_train, y_train)
print('Score: %.2f' % model.score(X_test, y_test))

# Decision Tree Classifier

In [ ]:
y = df['grade']
X = binaryValues
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
model = tree.DecisionTreeClassifier() 
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print('Score: %.2f' % score)

# Graphs

In [ ]:
plt.figure()
pd.concat([nominalValues, df['G3']], axis=1).boxplot(by="G3", figsize=(10, 10))
# df_norm[["freetime", "G3"]].boxplot(by="G3", figsize=(10, 5))
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
parallel_coordinates(pd.concat([nominalValues, df['grade']], axis=1), "grade",color=['b','r'])
plt.title('Parallel Coordinates Plot', fontsize=20, fontweight='bold')
plt.xlabel('Features', fontsize=15)
plt.ylabel('Features values', fontsize=15)
plt.legend(loc=1, prop={'size': 15}, frameon=True,shadow=True, facecolor="white", edgecolor="black")
plt.show()